Thực hiện chạy từ trên xuống dưới
sử dụng lệnh del <tên_model> để xóa model trước khi train

In [2]:
# Connect Driver
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Bước 1 - Add các thư viện cần thiết và khai báo Features

In [17]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

In [18]:
CSV_HEADER=['Flow Duration', 'Total Fwd Packet', 'Total Bwd packets',
       'Total Length of Fwd Packet', 'Total Length of Bwd Packet',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Bwd Packet Length Min',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Packet Length Min', 'Packet Length Max', 'Packet Length Mean',
       'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count',
       'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count',
       'URG Flag Count', 'CWR Flag Count', 'ECE Flag Count', 'Down/Up Ratio',
       'Average Packet Size', 'Fwd Segment Size Avg', 'Bwd Segment Size Avg',
       'Fwd Bytes/Bulk Avg', 'Fwd Packet/Bulk Avg', 'Fwd Bulk Rate Avg',
       'Bwd Bytes/Bulk Avg', 'Bwd Packet/Bulk Avg', 'Bwd Bulk Rate Avg',
       'Subflow Fwd Packets', 'Subflow Fwd Bytes', 'Subflow Bwd Packets',
       'Subflow Bwd Bytes', 'FWD Init Win Bytes', 'Bwd Init Win Bytes',
       'Fwd Act Data Pkts', 'Fwd Seg Size Min', 'Active Mean', 'Active Std',
       'Active Max', 'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max',
       'Idle Min', 'Label', 'attack_cat'] # bo Src_IP, Dst_IP, Timestamp, {'Flow ID', 'Src Port', 'Dst Port', 'Protocol'}
CSV_HEADER_WITHOUT_LABEL = ['Flow Duration', 'Total Fwd Packet', 'Total Bwd packets',
       'Total Length of Fwd Packet', 'Total Length of Bwd Packet',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Bwd Packet Length Min',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Packet Length Min', 'Packet Length Max', 'Packet Length Mean',
       'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count',
       'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count',
       'URG Flag Count', 'CWR Flag Count', 'ECE Flag Count', 'Down/Up Ratio',
       'Average Packet Size', 'Fwd Segment Size Avg', 'Bwd Segment Size Avg',
       'Fwd Bytes/Bulk Avg', 'Fwd Packet/Bulk Avg', 'Fwd Bulk Rate Avg',
       'Bwd Bytes/Bulk Avg', 'Bwd Packet/Bulk Avg', 'Bwd Bulk Rate Avg',
       'Subflow Fwd Packets', 'Subflow Fwd Bytes', 'Subflow Bwd Packets',
       'Subflow Bwd Bytes', 'FWD Init Win Bytes', 'Bwd Init Win Bytes',
       'Fwd Act Data Pkts', 'Fwd Seg Size Min', 'Active Mean', 'Active Std',
       'Active Max', 'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max',
       'Idle Min']

# Bảng mã hóa
encoded_to_original = {
    0: 'Benign',
    1: 'Analysis',
    2: 'Backdoor',
    3: 'DoS',
    4: 'Exploits',
    5: 'Fuzzers',
    6: 'Generic',
    7: 'Reconnaissance',
    8: 'Shellcode',
    9: 'Worms'
}
# decoded_labels = [encoded_to_original[label] for label in y_pred]
# decoded_labels

# Bước 2 - Đọc dữ liệu và chuẩn hóa dữ liệu

In [19]:
#df = pd.read_csv("/content/drive/MyDrive/BT3/CIC_UNSW-LABEL_Attack_cat.csv")
df = pd.read_csv('/content/drive/MyDrive/AnToanHeThongThongTin-CH2024/CIC_UNSW-LABEL_Attack_cat.csv')
df = df.drop(['attack_cat'], axis=1)

In [20]:
scaler = StandardScaler()
scaler.fit(df[CSV_HEADER_WITHOUT_LABEL])
df[CSV_HEADER_WITHOUT_LABEL] = scaler.transform(df[CSV_HEADER_WITHOUT_LABEL])

# Bước 3 - Chia dữ liệu - Chọn một trong 2 bước sau tương ứng

Chia dữ liệu và tách dữ liệu có nhãn là 4 ra để add vào tập train riêng.



## Bước 3.1 - Chi dữ liệu khi không train Label 4 - Exploits

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Chia dữ liệu thành train/test
train_data, test_data = train_test_split(df, test_size=0.2)
train_data_1, train_data_2 = train_test_split(train_data, test_size=0.5)

# Chia dữ liệu thành train/test
train_data, test_data = train_test_split(df, test_size=0.2)
# Lấy dữ liệu dựa trên giá trị cột Label
train_data_Label_4 = train_data[train_data['Label'] == 4]
train = train_data[train_data["Label"] != 4 ]

train_data_1, train_data_2 = train_test_split(train, test_size=0.5)
#train_data_2 = pd.concat([train_data_2, train_data_Label_4]) #add nhãn 4 vào train_data-2 tương ứng với model số 2
test_data.to_csv("test_data.csv", index=False, header=True)

# Xác định đặc trưng (X) và nhãn (y)
X_train_1 = train_data_1[CSV_HEADER_WITHOUT_LABEL]
y_train_1 = train_data_1['Label']

X_train_2 = train_data_2[CSV_HEADER_WITHOUT_LABEL]
y_train_2 = train_data_2['Label']

X_test = test_data[CSV_HEADER_WITHOUT_LABEL]
y_test = test_data['Label']

##Bước 3.2: Dành cho Train với Label 4 - Exploits

In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Chia dữ liệu thành train/test
train_data, test_data = train_test_split(df, test_size=0.2)
train_data_1, train_data_2 = train_test_split(train_data, test_size=0.5)

# Chia dữ liệu thành train/test
train_data, test_data = train_test_split(df, test_size=0.2)
# Lấy dữ liệu dựa trên giá trị cột Label
train_data_Label_4 = train_data[train_data['Label'] == 4]
train = train_data[train_data["Label"] != 4 ]

train_data_1, train_data_2 = train_test_split(train, test_size=0.5)
train_data_2 = pd.concat([train_data_2, train_data_Label_4]) #add nhãn 4 vào train_data-2 tương ứng với model số 2

# Lưu dữ liệu vào các tệp
# train_data_1.to_csv("train_data_1.csv", index=False, header=True)
# train_data_2.to_csv("train_data_2.csv", index=False, header=True)
# train_data.to_csv("train_data.csv", index=False, header=True)
test_data.to_csv("test_data.csv", index=False, header=True)

# Xác định đặc trưng (X) và nhãn (y)
X_train_1 = train_data_1[CSV_HEADER_WITHOUT_LABEL]
y_train_1 = train_data_1['Label']

X_train_2 = train_data_2[CSV_HEADER_WITHOUT_LABEL]
y_train_2 = train_data_2['Label']

X_test = test_data[CSV_HEADER_WITHOUT_LABEL]
y_test = test_data['Label']

# # Lưu dữ liệu vào các tệp
# train_data_1.to_csv("train_data_1.csv", index=False, header=True)
# train_data_2.to_csv("train_data_2.csv", index=False, header=True)
# #train_data.to_csv("train_data.csv", index=False, header=True)
# test_data.to_csv("test_data.csv", index=False, header=True)


## Bước 4 - Gán tập dữ liệu train và test cho X_train_1, X_train_2, X_test

Gán X_train cho train_data_1 và 2

In [22]:
# Xác định đặc trưng (X) và nhãn (y)
X_train_1 = train_data_1[CSV_HEADER_WITHOUT_LABEL]
y_train_1 = train_data_1['Label']

X_train_2 = train_data_2[CSV_HEADER_WITHOUT_LABEL]
y_train_2 = train_data_2['Label']

X_test = test_data[CSV_HEADER_WITHOUT_LABEL]
y_test = test_data['Label']


# Train model

##Bước 5 - Thực hiện train dữ liệu với 2 mô hình.

In [23]:
del model_1
del model_2
del model_avg

In [24]:
# Huấn luyện mô hình Logistic Regression cho client 1 và client 2
model_1 = LogisticRegression(max_iter=1000)
model_1.fit(X_train_1, y_train_1)

model_2 = LogisticRegression(max_iter=1000)
model_2.fit(X_train_2, y_train_2)


LogisticRegression(max_iter=1000)

## Bước 6 - Khởi tạo một mô hình Model_avg - để tính trọng số được train từ Model_1 và Model_2 theo cơ chế mean-based

In [25]:
# Lấy các trọng số từ mô hình của mỗi client
coef_1 = model_1.coef_
intercept_1 = model_1.intercept_

coef_2 = model_2.coef_
intercept_2 = model_2.intercept_
# Find the missing class
missing_class = set(model_2.classes_) - set(model_1.classes_)

# Add missing class to model_1 coef_ with 0 weights
for c in missing_class:
    coef_1 = np.vstack([coef_1, np.zeros((1, coef_1.shape[1]))])
    intercept_1 = np.append(intercept_1, 0)

# Now you should be able to average the coefficients
# avg_coef = (coef_1 + coef_2) / 2
# avg_intercept = (intercept_1 + intercept_2) / 2

In [26]:
# Tính kích thước của dữ liệu mỗi client
size_train_1 = len(X_train_1)
size_train_2 = len(X_train_2)

# Tính trọng số của mỗi client dựa trên kích thước dữ liệu
total_size = size_train_1 + size_train_2
weight_1 = size_train_1 / total_size
weight_2 = size_train_2 / total_size

# # Lấy các trọng số từ mô hình của mỗi client
# coef_1 = model_1.coef_
# intercept_1 = model_1.intercept_

# coef_2 = model_2.coef_
# intercept_2 = model_2.intercept_

# Tổng hợp trọng số theo công thức Weighted Federated Averaging
avg_coef = weight_1 * coef_1 + weight_2 * coef_2
avg_intercept = weight_1 * intercept_1 + weight_2 * intercept_2

# Tạo mô hình mới với trọng số tổng hợp
model_avg = LogisticRegression()
model_avg.coef_ = avg_coef
model_avg.intercept_ = avg_intercept
# Added line below to fix the error
model_avg.classes_ = model_1.classes_ # or model_2.classes_, they should be the same


In [27]:
import numpy as np
from sklearn.linear_model import LogisticRegression

# Lấy danh sách trọng số từ các client
coefs = [model_1.coef_, model_2.coef_]  # Thay đổi theo số lượng client
intercepts = [model_1.intercept_, model_2.intercept_]

# Ensure coefs have the same shape before calculating median
# Find the union of classes from both models
all_classes = sorted(list(set(model_1.classes_) | set(model_2.classes_)))

# Reshape coefs to include all classes, filling missing ones with 0
coefs_reshaped = []
for model_coef, model_classes in zip(coefs, [model_1.classes_, model_2.classes_]):
    coef_reshaped = np.zeros((len(all_classes), model_coef.shape[1]))
    for i, cls in enumerate(model_classes):
        coef_reshaped[all_classes.index(cls)] = model_coef[i]
    coefs_reshaped.append(coef_reshaped)

# Reshape intercepts to include all classes, filling missing ones with 0
intercepts_reshaped = []
for model_intercept, model_classes in zip(intercepts, [model_1.classes_, model_2.classes_]):
    intercept_reshaped = np.zeros(len(all_classes))
    for i, cls in enumerate(model_classes):
        intercept_reshaped[all_classes.index(cls)] = model_intercept[i]
    intercepts_reshaped.append(intercept_reshaped)

# Now calculate the median with reshaped coefficients
median_coef = np.median(coefs_reshaped, axis=0)
median_intercept = np.median(intercepts_reshaped, axis=0)

# Tạo mô hình mới với trọng số tổng hợp
model_avg = LogisticRegression()
model_avg.coef_ = median_coef
model_avg.intercept_ = median_intercept
model_avg.classes_ = np.array(all_classes)  # Set classes to the union of classes

# Bước 7 - Test model - Chọn một trong 2 test model tương ứng với train nhãn 4 và model không train nhãn 4

## Bước 7.1 - Thực hiện test Mô hình với nhãn số 4 đã được train

In [30]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Dự đoán với mô hình tổng hợp
y_pred = model_avg.predict(X_test)

# Đánh giá hiệu suất mô hình
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred))
recall = recall_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred))
f1 = f1_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred))

# In ra các chỉ số đánh giá
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score: {f1:.2f}")

# Tính Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(conf_matrix)

# In báo cáo phân loại (classification report)
report = classification_report(y_test, y_pred, target_names=encoded_to_original.values())
print("\nClassification Report:\n", report)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


Accuracy: 86.83%
Precision: 0.91
Recall: 0.87
F1-score: 0.86

Confusion Matrix:
[[69753     5     0    68    17  1643    15   147     0     0]
 [    3     8     0    82     0     6     0     0     0     0]
 [    4     0     6     2     8    66     2     4     0     0]
 [   25     7     1   252    14   430    62    60     0     0]
 [  221    11     0  2305   217  2704   540   155     0     0]
 [   20     2     0    12     9  5184    21   696     0     0]
 [   20     1     0    55     2   432   403    19     0     0]
 [   97     0     0     0     0  1314     0  1965     0     0]
 [    1     0     0     0     0   334     0   108     0     0]
 [    0     0     0     0     2    32     7     4     0     0]]

Classification Report:
                 precision    recall  f1-score   support

        Benign       0.99      0.97      0.98     71648
      Analysis       0.24      0.08      0.12        99
      Backdoor       0.86      0.07      0.12        92
           DoS       0.09      0.30    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Bước 7.2 - Thực hiện test mô hình khi dữ liệu nhãn số 4 chưa được train.

In [16]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Dự đoán với mô hình tổng hợp
y_pred = model_avg.predict(X_test)

# Đánh giá hiệu suất mô hình
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred))
recall = recall_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred))
f1 = f1_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred))

# In ra các chỉ số đánh giá
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score: {f1:.2f}")

# Tính Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(conf_matrix)

# In báo cáo phân loại (classification report)
report = classification_report(y_test, y_pred, target_names=encoded_to_original.values())
print("\nClassification Report:\n", report)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


Accuracy: 86.68%
Precision: 0.92
Recall: 0.94
F1-score: 0.92

Confusion Matrix:
[[69707     0     0    70     0  1631    16   178     0     0]
 [   20     0     0    54     0     2     4     0     0     0]
 [    7     0     5     5     0    61     1     5     0     0]
 [   44     0     5   290     0   456    67    61     0     0]
 [  397     0     0  2420     0  2572   469   312     0     0]
 [   25     0     0     7     0  5160    20   729     0     0]
 [   14     0     0    58     0   453   375    30     0     0]
 [    4     0     0     3     0  1273     0  2117     0     0]
 [    0     0     0     0     0   295     0   107     0     0]
 [    1     0     0     0     0    43     6     4     0     0]]

Classification Report:
                 precision    recall  f1-score   support

        Benign       0.99      0.97      0.98     71602
      Analysis       0.00      0.00      0.00        80
      Backdoor       0.50      0.06      0.11        84
           DoS       0.10      0.31    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
